<a href="https://colab.research.google.com/github/aud-z/cover_art_generation/blob/master/lyric_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
filepath = '/content/drive/MyDrive/Art_ML/songs_dataset_w_extract_summary.csv'

In [43]:
import pandas as pd
import nltk
import re 
import numpy as np

In [9]:
df = pd.read_csv(filepath)

In [10]:
df.head()

,uri,track,artist,cover image,lyrics,lyrics_cleaned,extract_summary
0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,...","Ask me if I give a fuck about a clique, ayy. A...","Ask me if I give a fuck about a diss, yuh. Ask..."
1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...,In due time it feels like this all makes sense...,"I’m tired of change, life has been weighing me..."
2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ...","Aye, Aye. Smoke a Dutch blunt. Don't flick the...",Giz I didn't fuckin mean it. Giz I didn't mean...
3,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra...","Ratatatatatata, boom!Ratatatatatata, boom!Two ...","Ratatatatatata, boom. And now we talking thric..."
4,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just...","Ayy, ayy, uh, uh, uh. See, I just can’t relate...","See, I just can’t relate. I can’t relate, I ca..."


In [116]:
df.drop_duplicates(subset=['uri'], inplace=True)

In [117]:
df.shape

(8109, 7)

In [118]:
# some songs don't have lyrics. We will exclude them from this analysis
df[df['lyrics_cleaned'].isna()]

,uri,track,artist,cover image,lyrics,lyrics_cleaned,extract_summary
1178,spotify:track:7bw3BR47CySVMrzZEuXsYl,METHADONE,deep october,https://i.scdn.co/image/ab67616d0000b273941cbb...,[To Be Transcribed],NaN,NaN
1797,spotify:track:6kBAlRKTukxlwmp0u8mde4,DMT,xxyyxx,https://i.scdn.co/image/ab67616d0000b2732487ef...,[Instrumental Track],NaN,NaN
1807,spotify:track:6qMVcqQtXBUsR1kQHIa2Qi,Ave Plague,king plague,https://i.scdn.co/image/ab67616d0000b273f3f2db...,[Music],NaN,NaN
1821,spotify:track:7rGI2jhDcnnfSJXXTpGDCV,Transcend,klimeks,https://i.scdn.co/image/ab67616d0000b273d2945e...,[Non-Lyrical Vocals],NaN,NaN
1831,spotify:track:2zbMHMeXUtcEUQzmRmOUra,Quatrain,sorrow,https://i.scdn.co/image/ab67616d0000b27306f099...,[Unintelligible],NaN,NaN
1832,spotify:track:7xPZePEWn8gNn4IgPz4fHn,Cherie,sorrow,https://i.scdn.co/image/ab67616d0000b27306f099...,[Unintelligible],NaN,NaN
1841,spotify:track:6EWrC23P3ZasH7cY2vxabe,First Dates,blvck ceiling,https://i.scdn.co/image/ab67616d0000b2734a957f...,[Intro][?][Instrumental][?],NaN,NaN
1842,spotify:track:46IsPPQjLoFpmrzmnBal26,Choppa,blvck ceiling,https://i.scdn.co/image/ab67616d0000b2734a957f...,[Intro][?][Instrumental],NaN,NaN
1843,spotify:track:526HQ5aYZu4LhBTUerx0w5,First Blood,blvck ceiling,https://i.scdn.co/image/ab67616d0000b2734a957f...,[Intro][Instrumental],NaN,NaN
1844,spotify:track:10ugMmxlkh8eLcvLgovDY9,All Wet,blvck ceiling,https://i.scdn.co/image/ab67616d0000b2734a957f...,[Intro][Instrumental],NaN,NaN


In [119]:
df1 = df[(df['lyrics_cleaned'].notna()) & (df['extract_summary'].notna())].copy()

In [120]:
df1.reset_index(inplace=True, drop=True)

In [121]:
df1['extract_summary'] = df1['extract_summary'].astype('str')

In [122]:
lyrics = df1['extract_summary'].values

### Clean text

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [15]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [16]:
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

In [95]:
def clean(text):
  text = str(text)
  text = text.strip() 
  text = text.lower()

  # remove special characters
  text = re.sub(r'\W', ' ', text) 

  # tokenize
  tokens = text.split()

  #lemmatize, remove stopwords
  tokens = [stemmer.lemmatize(str(i)) for i in tokens if i not in stops]

  return tokens

In [123]:
lyric_tokens = [clean(l) for l in lyrics]

In [124]:
lyric_tokens[9]

['fuck', 'yuh', 'yuh', 'fuck', 'meditate', 'fuck', 'la']

### FastText Embeddings

Fasttext word embeddings take into account character n-grams (i.e. enriches word vectors with subtext information). This allows for better performance on out-of-vocabulary words compared to other embedding methods. In lyrics, there will be a lot of misspelled or shortened words, so this embedding method may be better suited for this task. 

In [1]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3128481 sha256=cc89a54183f4d7412377c0049a3efa360ad4e2894323611c515f07d44128ad6a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [2]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [42]:
def get_vector(tokens):
  vec = np.zeros(300)
  for i in tokens:
    vec += ft.get_word_vector(i)
  vec /= len(tokens)
  return vec


In [125]:
lyric_vecs = [get_vector(l) for l in lyric_tokens]

In [126]:
lyric_vecs = np.array(lyric_vecs)
print(lyric_vecs.shape)

(8026, 300)


### Calculate similarity scores

#### Cosine similarity

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

In [127]:
similarity_mat = cosine_similarity(lyric_vecs)

In [128]:
similarity_mat.shape

(8026, 8026)

In [129]:
# for each song, get ID of top 10 ranked song indices
def get_top_10(similarity_matrix):
  top_10_indices = []
  for i in range(len(similarity_matrix)):
    scores = similarity_matrix[i]

    # the top similarity will be with the song itself, so we do not return that item in the top-10 list
    top_10 = np.argsort(scores)[::-1][1:11]

    top_10_indices.append(top_10)
  return top_10_indices

In [130]:
top_10_similarities = get_top_10(similarity_mat)

In [131]:
top_10_similarities = np.array(top_10_similarities)

In [132]:
top_10_similarities.shape

(8026, 10)

In [133]:
top_10_similarities[0]

array([3947, 5276, 4055, 5208, 1895, 3149, 3537, 1739, 1629,  934])

In [134]:
print('lyric:\n', df1.loc[0, 'extract_summary'])
print('\ntop 10 similar lyric summaries:\n')
for i in top_10_similarities[0]:
  print(df1.loc[i, 'extract_summary'])

lyric:
 Ask me if I give a fuck about a diss, yuh. Ask me if I give a fuck about fame, yuh

top 10 similar lyric summaries:

Fuck up out of my face like cryptickayik and then erase, yuh. Do not give a fuck about your feelings, not today, yuh
Ain't nothing different, fuck it. Yuh, right
Fuck her like I'm in a rush, yuh. I turn my voice to the books, yuh
I know she a goddess, yuh. I know that she got it, yuh
That bitch a baddie, yuh. And she call me daddy, yuh
, yuh. We testin' up, I'm like "Hold on" bitch
Yuh, yuh, ayy, I just wanna fuck shit up (It's up and it’s stuck). I just wanna fuck shit up (It's up and it’s stuck)
, someone, call a medic, yuh (Yuh).  like I'm on a mission, yuh (Yuh)
Yuh, yuh, yuh. Suicidal, you know how that thing go
Swear I wanna die, yeah, when you cross my—Said I wanna die, yuh, no, I'm not alright, yuh. I might start a riot, yuh, I'm so fuckin' tired, yuh


In [135]:
# to make sure the indices of songs match with the top-10 list later on, reset indices for df1 to preserve this information
df1.reset_index(inplace=True)

In [136]:
df1.head()

,index,uri,track,artist,cover image,lyrics,lyrics_cleaned,extract_summary
0,0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,...","Ask me if I give a fuck about a clique, ayy. A...","Ask me if I give a fuck about a diss, yuh. Ask..."
1,1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...,In due time it feels like this all makes sense...,"I’m tired of change, life has been weighing me..."
2,2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ...","Aye, Aye. Smoke a Dutch blunt. Don't flick the...",Giz I didn't fuckin mean it. Giz I didn't mean...
3,3,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra...","Ratatatatatata, boom!Ratatatatatata, boom!Two ...","Ratatatatatata, boom. And now we talking thric..."
4,4,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just...","Ayy, ayy, uh, uh, uh. See, I just can’t relate...","See, I just can’t relate. I can’t relate, I ca..."


In [138]:
df1['top_1_similar'] = top_10_similarities[:, 0]
df1['top_2_similar'] = top_10_similarities[:, 1]
df1['top_3_similar'] = top_10_similarities[:, 2]
df1['top_4_similar'] = top_10_similarities[:, 3]
df1['top_5_similar'] = top_10_similarities[:, 4]

In [139]:
np.savetxt('/content/drive/MyDrive/Art_ML/lyric_top_10_similarities.txt', top_10_similarities, delimiter=',')

In [141]:
df1.to_csv('/content/drive/MyDrive/Art_ML/songs_dataset_w_similarity.csv', index=False)